# Summarization : 

l'objectif de ce notebook est de tester differentes methodes pour extraire de l'information pertinante des scans : 

    - Filtre ( regex )
    - Summarization models (BART) [Non utilisé]
    
# ----------------------------------------

    - Input : 
            + Excel file : id + Text
    
    - Output : 
            + Excel file : {Id , number of frames, text SUMMARIZED )
    
    

In [18]:
# Libraries : 
import os
import pandas as pd
import re

In [19]:
path ='Output_OCR/'
# check all files in the folder
f_files_names = os.listdir(path)
f_files_names

['V1 output_ocr.xlsx', 'output_esyocr.xlsx']

In [20]:
pwd

'/home/isna@comptes.racine.local/ocr_enquetes_reclamations'

In [21]:
# read table of frame_VS_text
df_ocr = pd.read_excel(path + 'V1 output_ocr.xlsx')
df_ocr.head()

,Unnamed: 0,tiff_name,frames,text
0,0,{AF07E04C-73CE-48A5-B276-60BA692B3FB3}.tif,3,"Page 1\n\nFrom: ""BG DL SP Affiliations"" <Bg_dl..."
1,1,{6C8BBF86-2975-4FD0-8008-0124AC16310D}.tif,4,"Page 1\n\nFrom: ""BG DL SP PRESTA PREV"" <BG_DL_..."
2,2,{7048823A-B7DB-4455-A695-D473B235097D}.tif,3,"Attention, mail de gestion technique. Ce mail ..."
3,3,{767C5A35-657E-4430-9643-9C626B935F6B}.tif,2,"Page 1\n\nFrom: ""BG DL SP PRESTA PREV"" <BG_DL_..."
4,4,{2B3C2EFA-C6AE-4B99-A501-DA288AD1ABD9}.tif,2,"Page 1\n\nFrom: ""BG DL SP PRESTA PREV"" <BG_DL_..."


In [4]:
# Delete the '.tif' at the end of strings : 
df_ocr.tiff_name = df_ocr.tiff_name.apply(lambda x: x[:-4])

In [6]:
# check example :
print(df_ocr.iloc[5,3])
# print(df_ocr.iloc[5,2]) # for the new one

Page 1

From: "BG DL SP PRESTA PREV" <BG_DL_SP_PRESTA_PREV@ag2rlamondiale.fr>

Date: 3 Jan 2022 13:59:02 +0100

To: "BG_MAILS_SP_PRESTA_PREV_NUMERISATION"
<BG_MAILS_SP_PRESTA_PREV_NUMERISATION @ag2rlamondiale.fr>

Subject: TR: attente de complément de salaire suite a la demande de la SARL la Magiedes

Pains Avignon a mon égard

 

De : BG CG TOULOUSE PREST SANTE <cg.toulouse-prestsante@ag2rlamondiale.fr>

Envoyé : lundi 3 janvier 2022 11:17

A : BG CG CHARTRES Prestationsprevoyance <cg.chartres-prestationsprevoyance @ag2rlamondiale.fr>
Objet : attente de complément de salaire suite a la demande de la SARL la Magie des Pains Avignon a
mon égard

 

De : Magali Le Reverend <magali.lereverend@yahoo.fr>

Envoyé : samedi 1 janvier 2022 18:16

A: BG CG TOULOUSE PREST SANTE <cg.toulouse-prestsante@ae?rlamondiale fr>

Objet : attente de complément de salaire suite a la demande de la SARL la Magie des Pains Avignon a
mon égard

 

Bonjour,

Dans le cadre de mon arrét maladie (depuis le 6 Aoiit 

In [7]:
# read data :
# Classification_emails : df contenant l'id de chaque scan et son label (motifrecla)
df_class = pd.read_excel('Data/Classification_emails.xlsx')
df_class.head()

,IDDEMANDE,IDDOC,TYPEDEMANDE,NUMERO_CONTRAT,BRANCHE_CONTRAT,CONTRATINDIV,GARANTIE,MOTIFRECLA,REPONSERECLA
0,11295941,{AF07E04C-73CE-48A5-B276-60BA692B3FB3},Résiliation assuré,0JY4850,M,0JY4850,NaN,Manque de proactivité,COMPL_Favorable au client
1,11308265,{6C8BBF86-2975-4FD0-8008-0124AC16310D},Demande diverse - Presta Prev,0644491,P,0644491,INCAPACITE,Délai trop long,COMPL_Favorable au client
2,11323834,{7048823A-B7DB-4455-A695-D473B235097D},Demande diverse - AFFIL,0NT1218,M,0NT1218,NaN,Délai trop long,Défavorable au client
3,11309899,{767C5A35-657E-4430-9643-9C626B935F6B},Demande diverse - Presta Prev,2254335,P,2254335,CONGES EXCEPTIONNELS,Charge administrative,COMPL_Favorable au client
4,11311700,{2B3C2EFA-C6AE-4B99-A501-DA288AD1ABD9},Demande diverse - Presta Prev,0598865,P,0598865,INCAPACITE,Délai trop long,Défavorable au client


In [8]:
# check freq:
df_class.MOTIFRECLA.value_counts()

Délai trop long                         1643
Manque de fiabilité                      283
Charge administrative                    175
Absence de réponse directe au client     120
Manque de proactivité                    114
Complexité de langage                    102
Mauvaise qualité du contact               68
Incohérence de prix                       61
Problématiques web                        29
Difficulté à joindre le groupe            14
Name: MOTIFRECLA, dtype: int64

In [9]:
# merge the two df :
df = pd.merge(df_ocr, df_class[['IDDOC', 'MOTIFRECLA']],  how='left', left_on='tiff_name', right_on = 'IDDOC').drop(['IDDOC'],1).drop_duplicates()
df.head()

,Unnamed: 0,tiff_name,frames,text,MOTIFRECLA
0,0,{AF07E04C-73CE-48A5-B276-60BA692B3FB3},3,"Page 1\n\nFrom: ""BG DL SP Affiliations"" <Bg_dl...",Manque de proactivité
1,1,{6C8BBF86-2975-4FD0-8008-0124AC16310D},4,"Page 1\n\nFrom: ""BG DL SP PRESTA PREV"" <BG_DL_...",Délai trop long
2,2,{7048823A-B7DB-4455-A695-D473B235097D},3,"Attention, mail de gestion technique. Ce mail ...",Délai trop long
3,3,{767C5A35-657E-4430-9643-9C626B935F6B},2,"Page 1\n\nFrom: ""BG DL SP PRESTA PREV"" <BG_DL_...",Charge administrative
4,4,{2B3C2EFA-C6AE-4B99-A501-DA288AD1ABD9},2,"Page 1\n\nFrom: ""BG DL SP PRESTA PREV"" <BG_DL_...",Délai trop long


In [10]:
# check info 
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2609 entries, 0 to 2610
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  2609 non-null   int64 
 1   tiff_name   2609 non-null   object
 2   frames      2609 non-null   int64 
 3   text        2609 non-null   object
 4   MOTIFRECLA  2609 non-null   object
dtypes: int64(2), object(3)
memory usage: 122.3+ KB


## Clean data

In [11]:
# df = df.replace(r'\r+|\n+|\t+',' ', regex=True) # To remove carriage return (\r), new line (\n) and tab (\t)
# add lower() filter

## Create Filter

In [12]:
def Summarize_emails(email_message):
    '''
    Find rules (words/expression) to filter text
    
    Args:
        email_message: str
                       Text of scans
    
    Returns:
        summarized_email : List
                           List containing text summarized
        
        niveau_de_confiance: List
                             Rules used for summarization
    '''
    summarized_email = 'other method' # initialize OUTPUT with 'other_method' : look for more rules
    niveau_de_confiance = '0' # initialize level of confidence with '0' 

    # ( we are pretty sure of our rule here ) 
    if "client ainsi que ci-joint le document envoyé." in email_message:
        summarized_email = [x.split('Données pour traitement de la demande :') for x in email_message.split('client ainsi que ci-joint le document envoyé.')][1][0]
        niveau_de_confiance = 'High'

    # only used once - should delete maybe
    elif "Description de la requéte" in email_message:
      if 'Mail d\'origine' in email_message:
        summarized_email = [x.split('Description de la requéte') for x in email_message.split('Mail d\'origine')][1][0]
        niveau_de_confiance = 'High'

    # ( NOT as sure as we hope to be )
    else:
        try:
            
          keywords = ("Objet", "Sujet")
          kw_re = r'\b(?:' + '|'.join(map(re.escape, keywords)) + r')\b'
          match = (re.search(kw_re, email_message))
          print(match)
          if match != None :
              match = match.group()
              print('match group = ',match)

              if match == "Objet" :
                  if email_message.count("Objet")>1:
                      print("*** = ", email_message )
                      print(" ** ** *** taille de email : = ", ([x.split('Objet') for x in email_message.split('Objet')][1][0] ))
                      summarized_email = ''.join([x.split('Objet') for x in email_message.split('Objet ')][1][0].splitlines(keepends=True))
                      niveau_de_confiance = 'low'

              if len([x.split('Sujet') for x in email_message.split('Objet ')]) != 1:
                if match == "Sujet" :
                    if "Objet" in email_message:
                        summarized_email = ''.join([x.split('Sujet') for x in email_message.split('Objet ')][1][0].splitlines(keepends=True))
                        niveau_de_confiance = 'low'
        except:
            pass


    return [summarized_email, niveau_de_confiance]

#### niveau de confiance : 
    'High' : +++ information recherchée
    'Low' : + en moyenne, on obtient l'information recherchée
    '0': pas de texte (other method)

In [13]:
# clean text and add confidence_level
text_cleaned = [] # List to store cleaned text
confidence_level = [] # List to store classification of cleaned text
a = 0 # int for printing :  to check which scans returns an error
for line in df.text:
  print(a)
  resume = Summarize_emails(line)
  text_cleaned.append(resume[0])
  confidence_level.append(resume[1])
  a+= 1


0
1
<_sre.SRE_Match object; span=(891, 896), match='Objet'>
match group =  Objet
*** =  Page 1

From: "BG DL SP PRESTA PREV" <BG_DL_SP_PRESTA_PREV@ag2rlamondiale.fr>
Date: 3 Jan 2022 15:55:45 +0100

To: "BG_MAILS_SP_PRESTA_PREV_NUMERISATION"
<BG_MAILS_SP_PRESTA_PREV_NUMERISATION @ag2rlamondiale.fr>

Subject: TR: RELANCE : Dossier n° 03446542

Cordialement,

   

I), AG2R LA MONDIALE

Frederic GAILLARD (externe)
Gestionnaire des flux entrants

Direction du document multicanal (3DM)
485 avenue du Prado

13412 Marseille Cedex 20

Tel. +33 4 65 85 90 58

Découvrez tout AG2R LA MONDIALE

sur notre site et nos réseaux sociaux

wv

 

De: darty c.fardouet <cfardouet@orange.fr>

Envoyé : lundi 3 janvier 2022 14:56

A : BG CG RENNES GRECCO <cg-rennes.grecco @ag2rlamondiale.fr>;
compte.entreprise@ag2rlamondiale.fr; BG CG CHARTRES Prestationsprevoyance <cg.chartres-
prestationsprevoyance@ag2rlamondiale.fr>; BG CRC ADH COMPTE SP
<entreprise.santeprevoyance@ag2rlamondiale.fr>

Objet : RELANCE : Dos

In [14]:
# add columns :
df['text_cleaned'] = text_cleaned
df['confidence_level'] = confidence_level
df.head()

,Unnamed: 0,tiff_name,frames,text,MOTIFRECLA,text_cleaned,confidence_level
0,0,{AF07E04C-73CE-48A5-B276-60BA692B3FB3},3,"Page 1\n\nFrom: ""BG DL SP Affiliations"" <Bg_dl...",Manque de proactivité,"\n\nSuite 4 mon appel de ce jour , vous m'envo...",High
1,1,{6C8BBF86-2975-4FD0-8008-0124AC16310D},4,"Page 1\n\nFrom: ""BG DL SP PRESTA PREV"" <BG_DL_...",Délai trop long,": RELANCE : Dossier n° 03446542\n\nBonjour,\nP...",low
2,2,{7048823A-B7DB-4455-A695-D473B235097D},3,"Attention, mail de gestion technique. Ce mail ...",Délai trop long,"\n\nbonjour, Je n'ai toujours pas recu ma cart...",High
3,3,{767C5A35-657E-4430-9643-9C626B935F6B},2,"Page 1\n\nFrom: ""BG DL SP PRESTA PREV"" <BG_DL_...",Charge administrative,: Fwd: Congés exceptionnels ABDULLAH Aydin / A...,low
4,4,{2B3C2EFA-C6AE-4B99-A501-DA288AD1ABD9},2,"Page 1\n\nFrom: ""BG DL SP PRESTA PREV"" <BG_DL_...",Délai trop long,: RE:Votre demande concernant le dossier arrét...,low


In [15]:
# clean text:
df['text_cleaned'] = df['text_cleaned'].replace(r'\r+|\n+|\t+',' ', regex=True)
df.head()

,Unnamed: 0,tiff_name,frames,text,MOTIFRECLA,text_cleaned,confidence_level
0,0,{AF07E04C-73CE-48A5-B276-60BA692B3FB3},3,"Page 1\n\nFrom: ""BG DL SP Affiliations"" <Bg_dl...",Manque de proactivité,"Suite 4 mon appel de ce jour , vous m'envoyez...",High
1,1,{6C8BBF86-2975-4FD0-8008-0124AC16310D},4,"Page 1\n\nFrom: ""BG DL SP PRESTA PREV"" <BG_DL_...",Délai trop long,": RELANCE : Dossier n° 03446542 Bonjour, Pouve...",low
2,2,{7048823A-B7DB-4455-A695-D473B235097D},3,"Attention, mail de gestion technique. Ce mail ...",Délai trop long,"bonjour, Je n'ai toujours pas recu ma carte m...",High
3,3,{767C5A35-657E-4430-9643-9C626B935F6B},2,"Page 1\n\nFrom: ""BG DL SP PRESTA PREV"" <BG_DL_...",Charge administrative,: Fwd: Congés exceptionnels ABDULLAH Aydin / A...,low
4,4,{2B3C2EFA-C6AE-4B99-A501-DA288AD1ABD9},2,"Page 1\n\nFrom: ""BG DL SP PRESTA PREV"" <BG_DL_...",Délai trop long,: RE:Votre demande concernant le dossier arrét...,low


In [16]:
# freq of text not summarized:
text_cleaned.count('other method')

1521

In [17]:
# check freq :
df.confidence_level.value_counts()

0       1521
low      975
High     113
Name: confidence_level, dtype: int64

In [ ]:
df.to_excel( './Output_Summarization/OUTPUT_ocr_test2.xlsx', index=False, engine='xlsxwriter') # Download output









# THE END

#### Create sample : only 'High' ( to share for checking )

In [ ]:
df_high = df.loc[df['confidence_level'] == 'High']
# sample (35)
df_sampled = df_high.sample(n=35)
df_sampled.to_excel('OUTPUT_ocr_test_categorie_high.xlsx') # download output

### Summarization : (test)

In [52]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable


### Facebook bart large xsum

In [53]:
from transformers import pipeline
summarizer = pipeline("summarization" , model= 'facebook/bart-large-xsum')

Downloading:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.51G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [54]:
b = """Page 1\n\nFrom: "BG DL SP PRESTA PREV" <BG_DL_SP_PRESTA_PREV@ag2rlamondiale.fr>\n\nDate: 3 Jan 2022 13:59:02 +0100\n\nTo: "BG_MAILS_SP_PRESTA_PREV_NUMERISATION"\n<BG_MAILS_SP_PRESTA_PREV_NUMERISATION @ag2rlamondiale.fr>\n\nSubject: TR: attente de complément de salaire suite a la demande de la SARL la Magiedes\n\nPains Avignon a mon égard\n\n \n\nDe : BG CG TOULOUSE PREST SANTE <cg.toulouse-prestsante@ag2rlamondiale.fr>\n\nEnvoyé : lundi 3 janvier 2022 11:17\n\nA : BG CG CHARTRES Prestationsprevoyance <cg.chartres-prestationsprevoyance @ag2rlamondiale.fr>\nObjet : attente de complément de salaire suite a la demande de la SARL la Magie des Pains Avignon a\nmon égard\n\n \n\nDe : Magali Le Reverend <magali.lereverend@yahoo.fr>\n\nEnvoyé : samedi 1 janvier 2022 18:16\n\nA: BG CG TOULOUSE PREST SANTE <cg.toulouse-prestsante@ae?rlamondiale fr>\n\nObjet : attente de complément de salaire suite a la demande de la SARL la Magie des Pains Avignon a\nmon égard\n\n \n\nBonjour,\n\nDans le cadre de mon arrét maladie (depuis le 6 Aoiit et ce jusqu\'au 31 Décembre 2021) et étant affiliée a la\n\ncomplémentaire AG2R par le biais de mon employeur la SARL la Magie des Pains sur Avignon, la Barthelasse,\n\nce dernier cotise 4 un contrat de prévoyance qui permet au salarié en arrét de travail de bénéficier d\'un\n\ncomplément de salaire.\n\nA ce titre, mon employeur vous a contacté le 5 Novembre dernier pour vous en faire la demande. N\'ayant pas\n\nencore cu de retour de votre part, je reviens vers vous afin de savoir si vous pouviez m\'éclairer sur ce point- 1a.\n\nDans l\'attente de votre retour, je vous remercie.\n_x000C_Page 2\n\nBien a vous,\n\nMagali Le Révérend\n\n06.74.84.16.84\n_x000C """

In [55]:
print(summarizer(b, min_length = 15, do_sample= False))

[{'summary_text': "Dans le cadre de mon arrét maladie (depuis le 6 Aoiit et ce jusqu'au 31 Décembre 2021) and étant affiliée a la SARL la Magie des Pains Avignon, AG2R,"}]


In [ ]:
# to Test/add:
'mrm8488/camembert2camembert_shared-finetuned-french-summarization'
'facebook/bart-large-cnn'